# Comparing US and Canada Cities Project

## Importing Major City Lat and Long Excel File from Github (available for reference)

In [27]:
file = "https://github.com/AndrewKJacobs2/Coursera_Capstone/blob/master/US%20Latitudes%20and%20Longitudes%20NEW.xlsx?raw=true"
file

'https://github.com/AndrewKJacobs2/Coursera_Capstone/blob/master/US%20Latitudes%20and%20Longitudes%20NEW.xlsx?raw=true'

## Importing Libraries Needed For Project

In [28]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Read Excel File and Make Initial Map of US and Canada

In [29]:
citydata = pd.read_excel(file)

In [30]:
latitude = 41
longitude = -93

map_uscanada = folium.Map(location=[latitude, longitude], zoom_start=3)

# add markers to map
for lat, lng, label in zip(citydata['Latitude'], citydata['Longitude'], citydata['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_uscanada)  
    
map_uscanada

## Foursquare Credentials, Initial Venue Practice with Albany

In [31]:
CLIENT_ID = 'EGJ51MQNA4AD1YEPXGHZCK4VFATFLCK2BR0XSRNTYAR2VYPH' # your Foursquare ID
CLIENT_SECRET = '1KCUZYJJVTZO2MS4SHCV5SYKWJBYVNQOYD325X5W2NP51GEP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EGJ51MQNA4AD1YEPXGHZCK4VFATFLCK2BR0XSRNTYAR2VYPH
CLIENT_SECRET:1KCUZYJJVTZO2MS4SHCV5SYKWJBYVNQOYD325X5W2NP51GEP


In [32]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    42.652500, 
    -73.757220, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EGJ51MQNA4AD1YEPXGHZCK4VFATFLCK2BR0XSRNTYAR2VYPH&client_secret=1KCUZYJJVTZO2MS4SHCV5SYKWJBYVNQOYD325X5W2NP51GEP&v=20180605&ll=42.6525,-73.75722&radius=10000&limit=100'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2e5ec049b14d0023abd8a0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Albany',
  'headerFullLocation': 'Albany',
  'headerLocationGranularity': 'city',
  'totalResults': 237,
  'suggestedBounds': {'ne': {'lat': 42.74250009000009,
    'lng': -73.63507871822407},
   'sw': {'lat': 42.562499909999914, 'lng': -73.87936128177594}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55f6d343498edb711f07a4e1',
       'name': 'Renaissance Albany Hotel',
       'location': {'address': '144 State St',
        'crossStreet': 'Eagle Street',
        'lat': 42.65062486985366,
        'lng': -73.7556865811348,
       

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Renaissance Albany Hotel,Hotel,42.650625,-73.755687
1,Iron Gate Cafe,Café,42.655974,-73.762504
2,City Beer Hall,Pub,42.649660,-73.754787
3,The Olde English Pub & Pantry,Pub,42.653958,-73.748563
4,New York State Museum,Museum,42.648974,-73.761258


In [36]:
nearby_venues.tail()

,name,categories,lat,lng
95,Greenbush YMCA,Gym / Fitness Center,42.616988,-73.695576
96,Jim's Tastee-Freez,Ice Cream Shop,42.630996,-73.805044
97,Trader Joe's,Grocery Store,42.714444,-73.812380
98,Perfect Blend,Coffee Shop,42.622032,-73.832952
99,Bruegger's,Bagel Shop,42.628614,-73.812358


## Obtaining Venue Type Composition for All Cities Analyzed

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
uscanven = getNearbyVenues(names=citydata['City'],
                                   latitudes=citydata['Latitude'],
                                   longitudes=citydata['Longitude']
                                  )

Albany, N.Y.
Albuquerque, N.M.
Amarillo, Tex.
Anchorage, Alaska
Atlanta, Ga.
Austin, Tex.
Baker, Ore.
Baltimore, Md.
Bangor, Maine
Birmingham, Ala.
Bismarck, N.D.
Boise, Idaho
Boston, Mass.
Buffalo, N.Y.
Calgary, Alba., Can.
Carlsbad, N.M.
Charleston, S.C.
Charleston, W. Va.
Charlotte, N.C.
Cheyenne, Wyo.
Chicago, Ill.
Cincinnati, Ohio
Cleveland, Ohio
Columbia, S.C.
Columbus, Ohio
Dallas, Tex.
Denver, Colo.
Des Moines, Iowa
Detroit, Mich.
Dubuque, Iowa
Duluth, Minn.
Eastport, Maine
Edmonton, Alb., Can.
El Centro, Calif.
El Paso, Tex.
Eugene, Ore.
Fargo, N.D.
Flagstaff, Ariz.
Fort Worth, Tex.
Fresno, Calif.
Grand Junction, Colo.
Grand Rapids, Mich.
Havre, Mont.
Helena, Mont.
Honolulu, Hawaii
Hot Springs, Ark.
Houston, Tex.
Idaho Falls, Idaho
Indianapolis, Ind.
Jackson, Miss.
Jacksonville, Fla.
Juneau, Alaska
Kansas City, Mo.
Key West, Fla.
Kingston, Ont., Can.
Klamath Falls, Ore.
Knoxville, Tenn.
Las Vegas, Nev.
Lewiston, Idaho
Lincoln, Neb.
Long Beach, Calif.
Los Angeles, Calif.
Louisv

In [43]:
uscanven

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Albany, N.Y.",42.65250,-73.75722,Renaissance Albany Hotel,42.650625,-73.755687,Hotel
1,"Albany, N.Y.",42.65250,-73.75722,City Beer Hall,42.649660,-73.754787,Pub
2,"Albany, N.Y.",42.65250,-73.75722,The Egg,42.650593,-73.758390,Performing Arts Venue
3,"Albany, N.Y.",42.65250,-73.75722,Iron Gate Cafe,42.655974,-73.762504,Café
4,"Albany, N.Y.",42.65250,-73.75722,Public House 42,42.650171,-73.756492,English Restaurant
5,"Albany, N.Y.",42.65250,-73.75722,New York State Museum,42.648974,-73.761258,Museum
6,"Albany, N.Y.",42.65250,-73.75722,Cider Belly Donuts,42.650487,-73.752350,Café
7,"Albany, N.Y.",42.65250,-73.75722,Empire State Plaza,42.650492,-73.759504,Plaza
8,"Albany, N.Y.",42.65250,-73.75722,Palace Theatre,42.654736,-73.750192,Theater
9,"Albany, N.Y.",42.65250,-73.75722,Stacks Espresso Bar - Downtown,42.650257,-73.750645,Café


In [44]:
# one hot encoding
uscan_onehot = pd.get_dummies(uscanven[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
uscan_onehot['City'] = uscanven['City'] 

# move neighborhood column to the first column
fixed_columns = [uscan_onehot.columns[-1]] + list(uscan_onehot.columns[:-1])
uscan_onehot = uscan_onehot[fixed_columns]

uscan_onehot.head()

,City,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,...,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Listing Most Common Venues for Each City for Analysis

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = uscangrouped['City']

for ind in np.arange(uscangrouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(uscangrouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

## Dataset Used for Clustering; Share Scores for Each Type of Business by City

In [45]:
uscangrouped = uscan_onehot.groupby('City').mean().reset_index()
uscangrouped

,City,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,...,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Albany, N.Y.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.010000,0.01,0.00,0.000000,0.000000,0.010000,0.000000
1,"Albuquerque, N.M.",0.010000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,"Amarillo, Tex.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.015385,0.00,0.00,0.000000,0.000000,0.000000,0.000000
3,"Anchorage, Alaska",0.000000,0.020000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,"Atlanta, Ga.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.000000
5,"Austin, Tex.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.020000,0.000000
6,"Baker, Ore.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.025000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
7,"Baltimore, Md.",0.000000,0.000000,0.0,0.00,0.0,0.01,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,"Bangor, Maine",0.000000,0.000000,0.0,0.00,0.0,0.00,0.016667,0.033333,0.05,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
9,"Birmingham, Ala.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.016129,0.000000,0.000000,0.000000


In [46]:
uscangrouped.shape

(126, 431)